In [1]:
from picoscenes import Picoscenes

local_frames = Picoscenes("rx_n210_250310_094906.csi")
print("Number of local_frames:", len(local_frames.raw))

from scipy.io import loadmat
# Load the baseband signals from the .mat file
remote_baseband_data = loadmat('baseband_signals.mat')
remote_baseband_data = remote_baseband_data['baseband_data'][0]
print("Number of remote_baseband_data:", len(remote_baseband_data))

Number of local_frames: 117
Number of remote_baseband_data: 115


In [7]:
from numpy.testing import assert_array_equal
import hashlib

def find_matching_frames(local_frames, remote_data):
    # Create a hash map of local frames
    local_hash_map = {}
    for idx, frame in enumerate(local_frames.raw):
        if 'BasebandSignals' in frame:
            # Create a unique hash for the array data
            data = frame['BasebandSignals'].tobytes()
            h = hashlib.md5(data).hexdigest()
            local_hash_map.setdefault(h, []).append(idx)
    
    # Find matches in remote data
    matches = []
    unmatched_remote = []
    
    for remote_idx, remote_frame in enumerate(remote_data):
        remote_hash = hashlib.md5(remote_frame.tobytes()).hexdigest()
        if remote_hash in local_hash_map:
            for local_idx in local_hash_map[remote_hash]:
                try:
                    assert_array_equal(local_frames.raw[local_idx]['BasebandSignals'], remote_frame)
                    matches.append((local_idx, remote_idx))
                    break
                except AssertionError:
                    continue
            else:
                unmatched_remote.append(remote_idx)
        else:
            unmatched_remote.append(remote_idx)
    
    return matches, unmatched_remote

# Usage
matches, unmatched = find_matching_frames(local_frames, remote_baseband_data)

print(f"Found {len(matches)} matching frame pairs:")
for local_idx, remote_idx in matches:
    print(f"Remote frame {remote_idx} matches local frame {local_idx}")

print(f"\n{len(unmatched)} remote frames had no matches:")
print(unmatched)

Found 115 matching frame pairs:
Remote frame 0 matches local frame 0
Remote frame 1 matches local frame 1
Remote frame 2 matches local frame 2
Remote frame 3 matches local frame 3
Remote frame 4 matches local frame 4
Remote frame 5 matches local frame 5
Remote frame 6 matches local frame 6
Remote frame 7 matches local frame 7
Remote frame 8 matches local frame 8
Remote frame 9 matches local frame 9
Remote frame 10 matches local frame 10
Remote frame 11 matches local frame 11
Remote frame 12 matches local frame 12
Remote frame 13 matches local frame 13
Remote frame 14 matches local frame 14
Remote frame 15 matches local frame 15
Remote frame 16 matches local frame 16
Remote frame 17 matches local frame 17
Remote frame 18 matches local frame 18
Remote frame 19 matches local frame 19
Remote frame 20 matches local frame 20
Remote frame 21 matches local frame 21
Remote frame 22 matches local frame 22
Remote frame 23 matches local frame 24
Remote frame 24 matches local frame 25
Remote frame 